In [78]:
import pandas as pd
import numpy as np
from IPython.display import display
import re
import time
import json

In [79]:
# Инфраструктура
news_df = pd.read_csv('data/MVD_News/news_with_location.csv')
objects_df = pd.read_csv('data/OpenData/Объекты недвижимого имущества и земельные участки.csv')
houses_df = pd.read_csv('data/OpenData/Технико-экономические паспорта многоквартирных домов.csv')
amenities_df = pd.read_csv('data/OpenStreetMap/amenities.csv')
shops_df = pd.read_csv('data/OpenStreetMap/shops.csv')

# проблемы города
problems_df = pd.read_csv('data/GorodGovSpb/GorodGovSpb.csv')

In [80]:
display(news_df.info())
display(news_df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6140 entries, 0 to 6139
Data columns (total 6 columns):
id            6140 non-null int64
title         6140 non-null object
time          6140 non-null object
lat           6140 non-null float64
lng           6140 non-null float64
decreepart    6140 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 287.9+ KB


None

,id,title,time,lat,lng,decreepart
0,11624017,В Красносельском районе гражданин привлечен к ...,2017-11-20 13:07:00,59.836513,30.133829,ст.20.8 (Нарушение Правил Хранения Оружия и Па...
1,11620224,"Полиция разыскивает злоумышленника, ограбившег...",2017-11-20 08:35:00,59.998492,29.759626,ч.2 ст.161 (Грабеж); Уголовный Кодекс; Россия
2,11617626,В Ленобласти задержан подозреваемый в хищении ...,2017-11-19 10:48:00,60.076238,30.121383,ч.2 ст.158 (Кража); Уголовный Кодекс; Россия
3,11617623,В Петербурге задержан автоугонщик,2017-11-19 10:45:00,59.861774,30.174635,ч.1 ст.166 (Неправомерное Завладение Автомобил...
4,11606722,В Санкт-Петербурге задержаны подозреваемые в х...,2017-11-17 11:23:00,59.972558,30.304503,"ч.2 ст.164 (Хищение Предметов, Имеющих Особую ..."
5,11604535,В Красносельском районе Петербурга по «горячим...,2017-11-17 08:27:00,59.859373,30.211193,ч.3 ст.162 (Разбой); Уголовный Кодекс; Россия
6,11594536,В Ленинградской области по «горячим» следам за...,2017-11-16 08:46:00,60.076238,30.121383,ч.2 ст.162 (Разбой); Уголовный Кодекс; Россия
7,11568968,В Петербурге задержаны подозреваемые в похищен...,2017-11-13 13:02:00,59.717593,30.406004,ч.2 ст.126 (Похищение Человека); Уголовный Код...
8,11565028,В Петроградском районе Петербурга раскрыто раз...,2017-11-13 09:07:00,59.959253,30.315649,ч.2 ст.162 (Разбой); Уголовный Кодекс; Россия
9,11565011,Полиция расследует ограбление граждан в Красно...,2017-11-13 09:06:00,59.852444,30.140841,ст.161 (Грабеж); Уголовный Кодекс; Россия


In [81]:
for year in range(2010, 2018):
    min_weight = None
    weighted_points = []
    for index, row in news_df.iterrows():
        cur_year = int(row['time'].split('-')[0])
        if cur_year > year:
            continue
        decreepart = row['decreepart']
        rule = decreepart.split(';')[0]
        rule_name_match = re.match(r'.*\((\w*).*\).*', rule)
        if rule_name_match is None:
            name = ''
        else:
            name = rule_name_match.groups(0)[0].lower().split()[0]
        if name == 'убийство':
            weight = 2
        elif name == 'грабеж' or name == 'умышленное' or name == 'разбой':
            weight = 1.5
        else:
            weight = 1

        if min_weight is None:
            min_weight = weight
        weight /= year - cur_year + 1
        weighted_points.append({'lat': row.lat, 'lng': row.lng, 'weight': weight})
    for dict_ in weighted_points:
        dict_['weight'] /= min_weight
    with open('{}.json'.format(year), 'w') as file_to:
        json.dump(weighted_points, file_to)

In [82]:
import pandas as pd

level_of_criminal = pd.DataFrame(index=news_df.index, columns=['year', 'lat', 'lng', 'weight'])
for index, row in news_df.iterrows():
    cur_year = int(row['time'].split('-')[0])
    if cur_year > year:
        continue
    decreepart = row['decreepart']
    rule = decreepart.split(';')[0]
    rule_name_match = re.match(r'.*\((\w*).*\).*', rule)
    if rule_name_match is None:
        name = ''
    else:
        name = rule_name_match.groups(0)[0].lower().split()[0]
    if name == 'убийство':
        weight = 2
    elif name == 'грабеж' or name == 'умышленное' or name == 'разбой':
        weight = 1.5
    else:
        weight = 1

    weight /= year - cur_year + 1
    level_of_criminal.loc[index] = [cur_year, row.lat, row.lng, weight]
    

In [83]:
level_of_criminal.head()

,year,lat,lng,weight
0,2017,59.8365,30.1338,1
1,2017,59.9985,29.7596,1.5
2,2017,60.0762,30.1214,1
3,2017,59.8618,30.1746,1
4,2017,59.9726,30.3045,1


In [84]:
def get_data(year):
    data = pd.DataFrame(index = houses_df.index, columns=['people', 'years', 'square', 'floats', 'steps', 'houses', 'amenity', 'shops', 'criminal'])

    for index, row in houses_df.iterrows():
        print(index)

        lat = row['lat']
        lng = row['lng']

        year_criminal = level_of_criminal[level_of_criminal.year == year]

        year_location_criminal = year_criminal[(year_criminal.lat - lat)**2 + (year_criminal.lng - lng)**2 <= 0.0035]

        value_criminal = (year_location_criminal.weight/((year_location_criminal.lat - lat)**2 + (year_location_criminal.lng - lng)**2 + 1)).mean()

        location_amenities = amenities_df[(amenities_df.lat - lat)**2 + (amenities_df.lon - lng)**2 <= 0.0035]
        value_amenities = (1/((location_amenities.lat - lat)**2 + (location_amenities.lon - lng)**2 + 1)).mean()

        location_shops = shops_df[(shops_df.lat - lat)**2 + (shops_df.lon - lng)**2 <= 0.0035]
        value_shops = 1/((location_shops.lat - lat)**2 + (location_shops.lon - lng)**2 + 1).mean()
        
        location_houses_df = houses_df[(houses_df.lat - lat)**2 + (houses_df.lng - lng)**2 <= 0.0035]
        values_houses = (1/((location_houses_df.lat - lat)**2 + (location_houses_df.lng - lng)**2 + 1)).mean()

        years = year - int(row['Год постройки'].split('-')[0])
        data.loc[index, 'people'] = row['Количество проживающих']
        data.loc[index, 'years'] = years 
        data.loc[index, 'square'] = row['Общая площадь здания']
        data.loc[index, 'floats'] = row['Число этажей']
        data.loc[index, 'steps'] = row['Число лестниц']
        data.loc[index, 'houses'] = values_houses
        data.loc[index, 'amentity'] = value_amenities
        data.loc[index, 'shops'] = value_shops
        data.loc[index, 'criminal'] = value_criminal
        
    return data

In [ ]:
year = 2017

data = get_data(year)

In [86]:
data['criminal'] = data.criminal/data.criminal.min()

In [87]:
data['criminal'] = data.criminal.fillna(0)
data['shops'] = data.shops.fillna(0)
data['amenity'] = data.amenity.fillna(0)
data_without_na = data.fillna(data.mean())
data.head()

,people,years,square,floats,steps,houses,amenity,shops,criminal,amentity
0,213,50,3601.45,5,7,0.99867,0,0.998646,1.145450,0.998625
1,69,116,1581.38,6,2,0.998286,0,0.998206,1.135104,0.998230
2,82,52,2334,9,1,0.998437,0,0.998546,1.155062,0.998532
3,60,101,1978.15,5,1,0.998124,0,0.998307,1.128628,0.998255
4,344,117,8968.5,3,4,0.997497,0,0.998171,1.000008,0.998272


In [ ]:
year = 2014
data_learn = get_data(year)

In [41]:
data_learn['criminal'] = data_learn.criminal/data_learn.criminal.min()

In [42]:
data_learn['criminal'] = data_learn.criminal.fillna(0)
data_learn['shops'] = data_learn.shops.fillna(0)
data_learn['amenity'] = data_learn['amenity'].fillna(0)
data_learn_without_na = data_learn.fillna(data_learn.mean())

In [43]:
X_learn = data_learn_without_na[['people', 'years', 'square', 'floats', 'steps', 'houses', 'amenity', 'shops']]
y_learn = data_learn_without_na['criminal']
X_test = data_without_na[['people', 'years', 'square', 'floats', 'steps', 'houses', 'amenity', 'shops']]
y_test = data_without_na['criminal']

In [44]:
def L2(vec1, vec2):
    return ((vec1 - vec2)**2).sum()

### Linear regression

In [45]:
from sklearn.linear_model import Lasso
r = Lasso(alpha=0.1)
r.fit(X_learn, y_learn)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [46]:
linear_predict = r.predict(X_test)
L2(y_test, linear_predict)

401.5191545295163

### SVM

In [47]:
from sklearn.svm import SVR
svc = SVR()
svc.fit(X_learn, y_learn)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [48]:
svc_predict = svc.predict(X_test)
L2(y_test, svc_predict)

708.38648711056157

### Trees

In [49]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(X_learn, y_learn)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [50]:
tree_predict = tree.predict(X_test)
L2(y_test, tree_predict)

458.76135942366921

### XGBoost

In [51]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor()
clf.fit(X_learn, y_learn)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [52]:
clf_predict = clf.predict(X_test)
L2(y_test, clf_predict)

346.80600485508182

### RandomForest

In [53]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_learn, y_learn)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [54]:
rfr_predict = rfr.predict(X_test)
L2(y_test, rfr_predict)

388.01799224974445

### Ensemble

In [55]:
ansemle_list = [linear_predict, svc_predict, clf_predict, rfr_predict, tree_predict]
ansemble_predict = sum(ansemle_list)/len(ansemle_list)
L2(y_test, ansemble_predict)

396.60016264330386

### Predictions

In [ ]:
year = 2017

data_learn = get_data(year)

In [57]:
data_learn['criminal'] = data_learn.criminal/data_learn.criminal.min()

In [58]:
data_learn['criminal'] = data_learn.criminal.fillna(0)
data_learn['shops'] = data_learn.shops.fillna(0)
data_learn['amenity'] = data_learn.amenity.fillna(0)
data_learn_without_na = data_learn.fillna(data_learn.mean())
data.head()

,people,years,square,floats,steps,houses,amenity,shops,criminal,amentity
0,213,50,3601.45,5,7,0.99867,0,0.998646,1.145450,0.998625
1,69,116,1581.38,6,2,0.998286,0,0.998206,1.135104,0.998230
2,82,52,2334,9,1,0.998437,0,0.998546,1.155062,0.998532
3,60,101,1978.15,5,1,0.998124,0,0.998307,1.128628,0.998255
4,344,117,8968.5,3,4,0.997497,0,0.998171,1.000008,0.998272


In [59]:
X_learn = data_learn_without_na[['people', 'years', 'square', 'floats', 'steps', 'houses', 'amenity', 'shops']]
y_learn = data_learn_without_na['criminal']

In [60]:
from sklearn.linear_model import Lasso
r = Lasso(alpha=0.1)
r.fit(X_learn, y_learn)

from sklearn.svm import SVR
svc = SVR()
svc.fit(X_learn, y_learn)

from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(X_learn, y_learn)

from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor()
clf.fit(X_learn, y_learn)

from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_learn, y_learn)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
year = 2018
data = get_data(year)

In [62]:
data['criminal'] = data.criminal/data.criminal.min()

In [63]:
data['criminal'] = data.criminal.fillna(0)
data['shops'] = data.shops.fillna(0)
data['amenity'] = data['amentity'].fillna(0)
data_without_na = data.fillna(data.mean())
data.head()

,people,years,square,floats,steps,houses,amenity,shops,criminal,amentity
0,213,51,3601.45,5,7,0.99867,0.998625,0.998646,0,0.998625
1,69,117,1581.38,6,2,0.998286,0.998230,0.998206,0,0.998230
2,82,53,2334,9,1,0.998437,0.998532,0.998546,0,0.998532
3,60,102,1978.15,5,1,0.998124,0.998255,0.998307,0,0.998255
4,344,118,8968.5,3,4,0.997497,0.998272,0.998171,0,0.998272


In [64]:
X_test = data_without_na[['people', 'years', 'square', 'floats', 'steps', 'houses', 'amenity', 'shops']]

In [66]:
ensemle_list = [r.predict(X_test), svc.predict(X_test), clf.predict(X_test), rfr.predict(X_test), tree.predict(X_test)]
ensemble_predict = sum(ensemle_list)/len(ensemle_list)

In [70]:
weighted_points = []
for index, row in X_test.iterrows():
    weighted_points.append({'lat': houses_df.lat[index], 'lng': houses_df.lng[index], 'weight': ensemble_predict[index]})
with open('{}.json'.format(year), 'w') as file_to:
    json.dump(weighted_points, file_to)

In [ ]:
year = 2019
data = get_data(year)

In [72]:
data['criminal'] = data.criminal/data.criminal.min()

In [73]:
data['criminal'] = data.criminal.fillna(0)
data['shops'] = data.shops.fillna(0)
data['amenity'] = data['amentity'].fillna(0)
data_without_na = data.fillna(data.mean())
data.head()

,people,years,square,floats,steps,houses,amenity,shops,criminal,amentity
0,213,52,3601.45,5,7,0.99867,0.998625,0.998646,0,0.998625
1,69,118,1581.38,6,2,0.998286,0.998230,0.998206,0,0.998230
2,82,54,2334,9,1,0.998437,0.998532,0.998546,0,0.998532
3,60,103,1978.15,5,1,0.998124,0.998255,0.998307,0,0.998255
4,344,119,8968.5,3,4,0.997497,0.998272,0.998171,0,0.998272


In [74]:
X_test = data_without_na[['people', 'years', 'square', 'floats', 'steps', 'houses', 'amenity', 'shops']]

In [76]:
ensemle_list = [r.predict(X_test), svc.predict(X_test), clf.predict(X_test), rfr.predict(X_test), tree.predict(X_test)]
ensemble_predict = sum(ensemle_list)/len(ensemle_list)

In [77]:
weighted_points = []
for index, row in X_test.iterrows():
    weighted_points.append({'lat': houses_df.lat[index], 'lng': houses_df.lng[index], 'weight': ensemble_predict[index]})
with open('{}.json'.format(year), 'w') as file_to:
    json.dump(weighted_points, file_to)